In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import joblib

# Suppress specific warnings from Keras and other libraries
warnings.filterwarnings('ignore', category=UserWarning, module='keras')

# Load the data from a CSV file
data = pd.read_csv('PST_v6.csv')  # Replace 'topography.csv' with the actual file name

# Remove commas and convert to numeric values
data = data.replace(',', '', regex=True).apply(pd.to_numeric, errors='coerce')
data = data.dropna()

# Define input (X) and output (y) variables
X = data[['cpower1', 'cspeed1', 'cpower2','cspeed2','angle']]
y = data[['eta_SID', 'rho_s_SID', 'sig_s_SID', 'Sa', 'Sq', 'Ssk', 'Sku', 'sigma_s_rho','Sdq','Sdr']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameter grid for grid search
param_grid = {
    'hidden_layer_sizes': [(50, 50, 50), (100, 100, 100), (50, 100, 50)],
    'activation': ['tanh', 'relu'],
    'learning_rate': [0.0001, 0.001, 0.01]
}

# Function to calculate normalized RMSE and MAE
def normalized_rmse_mae(y_true, y_pred):
    range_actual = y_true.max() - y_true.min()  # Range of actual values for normalization
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = np.mean(np.abs(y_true - y_pred))
    
    # Normalize RMSE and MAE
    normalized_rmse = rmse / range_actual
    normalized_mae = mae / range_actual
    
    return normalized_rmse, normalized_mae

# Custom wrapper class to fully implement scikit-learn's API
class KerasModelWrapper:
    def __init__(self, hidden_layer_sizes=(50, 100, 50), activation='relu', learning_rate=0.001, epochs=200, batch_size=32):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.history = None

    def build_model(self, input_dim):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(input_dim,)))
        for units in self.hidden_layer_sizes:
            model.add(tf.keras.layers.Dense(units, activation=self.activation, kernel_regularizer=l2(0.001)))  # L2 Regularization
        model.add(tf.keras.layers.Dense(1))  # Output layer for regression
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='mean_squared_error')
        return model

    def fit(self, X, y):
        input_dim = X.shape[1]
        self.model = self.build_model(input_dim)
        
        # Fit the model and store the history
        self.history = self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0, validation_split=0.1)
        return self

    def predict(self, X):
        return self.model.predict(X).flatten()

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    def save(self, filename):
        # Save the model as a .pkl file using joblib
        joblib.dump(self, filename)

    @staticmethod
    def load(filename):
        # Load the model from a .pkl file
        return joblib.load(filename)

# 10-fold Cross-validation setup
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Loop through each target variable
for i, col in enumerate(y.columns):
    
    best_r2 = -np.inf
    best_model = None
    best_params = {}

    for hidden_layer_sizes in param_grid['hidden_layer_sizes']:
        for activation in param_grid['activation']:
            for learning_rate in param_grid['learning_rate']:
                
                # Perform cross-validation manually
                cv_r2_scores = []
                
                for train_index, test_index in kf.split(X_train_scaled):
                    X_train_fold, X_test_fold = X_train_scaled[train_index], X_train_scaled[test_index]
                    y_train_fold, y_test_fold = y_train.iloc[train_index, i], y_train.iloc[test_index, i]

                    # Train and evaluate the model
                    model = KerasModelWrapper(hidden_layer_sizes=hidden_layer_sizes, activation=activation, learning_rate=learning_rate)
                    model.fit(X_train_fold, y_train_fold)
                    
                    # Evaluate the model on the test fold
                    y_pred_fold = model.predict(X_test_fold)
                    r2_fold = r2_score(y_test_fold, y_pred_fold)
                    
                    cv_r2_scores.append(r2_fold)
                
                # Average cross-validated R²
                mean_r2 = np.mean(cv_r2_scores)
                if mean_r2 > best_r2:
                    best_r2 = mean_r2
                    best_model = model
                    best_params = {
                        'hidden_layer_sizes': hidden_layer_sizes,
                        'activation': activation,
                        'learning_rate': learning_rate
                    }

    # Final evaluation on the test set
    y_pred = best_model.predict(X_test_scaled)
    r2 = best_model.score(X_test_scaled, y_test.iloc[:, i])
    normalized_rmse, normalized_mae = normalized_rmse_mae(y_test.iloc[:, i], y_pred)

    # Suppress intermediate output and only show relevant metrics and plots
    print(f"{col} - R² Score: {r2:.4f}, Normalized RMSE: {normalized_rmse:.4f}, Normalized MAE: {normalized_mae:.4f}")

    # Plot the training and validation loss
    plt.figure(figsize=(10, 6))
    plt.plot(best_model.history.history['loss'], label='Training Loss')
    plt.plot(best_model.history.history['val_loss'], label='Validation Loss', linestyle='--')
    plt.xlabel('Epoch', fontsize=20)
    plt.ylabel('Loss', fontsize=20)
    plt.title(f'Training and Validation Loss for {col}', fontsize=20)
    plt.legend(fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.show()

    # Plot Actual vs Predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test.iloc[:, i], y_pred, color='blue')
    plt.plot([y_test.iloc[:, i].min(), y_test.iloc[:, i].max()], 
             [y_test.iloc[:, i].min(), y_test.iloc[:, i].max()], color='red', lw=2)
    plt.xlabel(f'Actual {col}', fontsize=20)
    plt.ylabel(f'Predicted {col}', fontsize=20)
    plt.title(f'Actual vs Predicted for {col}', fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.show()

    # Calculate Permutation Feature Importance (PFI) using the original input features
    pfi = permutation_importance(best_model, X_test_scaled, y_test.iloc[:, i], n_repeats=10, random_state=42)

    # Plot the PFI results using the original input features
    feature_importances = pd.Series(pfi.importances_mean, index=X.columns)
    feature_importances = feature_importances.sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances.index, feature_importances.values)
    plt.title(f'Permutation Feature Importance for {col}', fontsize=20)
    plt.xlabel('Mean Importance Score', fontsize=18)
    plt.ylabel('Feature', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.show()


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
{
    "python.defaultInterpreterPath": "C:\\Users\\<YourUsername>\\AppData\\Local\\Programs\\Python\\Python312\\python.exe"
}


{'python.defaultInterpreterPath': 'C:\\Users\\<YourUsername>\\AppData\\Local\\Programs\\Python\\Python312\\python.exe'}